In [1]:
#Importamos las librerías que vamos a necesitar
import pandas as pd
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import dotenv
dotenv.load_dotenv()

True

In [3]:
# Librería para trabajar con bases de datos SQL
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [ ]:
# Librerías para graficar
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Antes de meternos de lleno en las consultas y en las gráficas recordemos los objetivos del proyecto, para así poner en contexto nuestro análisis

## Objetivo del analisis

**1. Comparación de Precios entre Supermercados:** Determinar qué supermercados ofrecen los precios más bajos y cuáles son más caros para cada producto.

**2. Análisis de la Evolución de Precios:** Estudiar cómo han cambiado los precios de los productos a lo largo del tiempo en distintos supermercados.

**3. Detección de Anomalías:** Identificar subidas o bajadas de precios inusuales que podrían señalar prácticas abusivas o promociones.

**4. Análisis de la Dispersión de Precios:** Evaluar la variabilidad de los precios de un mismo producto en diferentes supermercados.

**5. Comparación de Precios Promedio:** Calcular y comparar los precios promedio de cada producto en diferentes supermercados.

Procedemos con nuestras consultas y visualizaciones de acuerdo a estos puntos.

In [4]:
key = os.getenv("token")

In [5]:
#En primer lugar nos conectamso a la base de datos

def conexion_a_dbeaver(database):
    """
    Establece una conexión a una base de datos DBeaver.

    Args:
        database (str): El nombre de la base de datos.

    Returns:
        conexion: Un objeto de conexión a la base de datos.
    """
    try:
        conexion = psycopg2.connect(
            database=database,
            user="postgres",
            password=key,
            host="localhost",
            port="5432"
        )
    except OperationalError as e:
        if e.pgcode == errorcodes.INVALID_PASSWORD:
            print("La contraseña intorducida es errónea")
        elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
            print("Se ha producido un error de conexión")
        else:
            print(f"Ocurrió el error {e}")
    return conexion

In [6]:
conexion=conexion_a_dbeaver("supermercados")
cursor=conexion.cursor()

1.Evaluar la variabilidad de los precios de un mismo producto en diferentes supermercados.

In [7]:
query1 = """
SELECT 
    s.nombre AS supermercado,
    tp.nombre AS producto,
    MIN(h."precio") AS precio_minimo,
    MAX(h."precio") AS precio_maximo
FROM 
    historico h
INNER JOIN 
    tipo_producto tp ON h.id_producto = tp.id_producto
INNER JOIN 
    supermercado s ON h.id_super = s.id_super
GROUP BY 
    s.nombre, tp.nombre
ORDER BY 
    tp.nombre, precio_minimo;
"""

In [8]:
cursor.execute(query1)

In [9]:
resultado1 = cursor.fetchall()
df_query1 = pd.DataFrame(resultado1)
df_query1.columns = [col[0] for col in cursor.description]


In [10]:
df_query1

,supermercado,producto,precio_minimo,precio_maximo
0,alcampo,aceite de girasol,1.00,15.88
1,carrefour,aceite de girasol,1.00,15.88
2,dia,aceite de girasol,1.00,15.88
3,eroski,aceite de girasol,1.00,15.88
4,hipercor,aceite de girasol,1.00,15.88
5,mercadona,aceite de girasol,1.00,15.88
6,alcampo,aceite de oliva,2.83,62.99
7,carrefour,aceite de oliva,2.83,62.99
8,dia,aceite de oliva,2.83,62.99
9,eroski,aceite de oliva,2.83,62.99
